# <font color="purple">**Licenciatura en Tecnologías para la Información en Ciencias**</font>

## <font color="blue">**Procesamiento Digital de Imágenes**</font>

### <font color="blue">**Proyecto: Segmentación**</font>

### Angélica Nayeli Rivas Bedolla:    418128734
### Pablo Clemente Moreno:   418125805

### Semestre 2021-1

### Viernes 05 de febrero de 2021

# Introducción

La visión por computadora (en inglés Computer Vision) es un campo científico interdisciplinario que se ocupa de cómo pueden hacer las computadoras para obtener una comprensión de alto nivel a partir de imágenes o videos digitales. Desde la perspectiva de la ingeniería, busca automatizar las tareas que puede realizar el sistema visual humano.<br>

$\hspace{11cm}$ <img src="./computer_vision.jpeg"/><br>

Para entender qué es la __segmentación__ semántica existen varios niveles de granularidad en los que las computadoras pueden comprender las imágenes. Para cada uno de estos niveles hay un problema definido en el dominio de Computer Vision:<br>
$\hspace{7cm}$<img src="./intro_img.png" width="512"/>


# Enunciado del problema

Diseñar e implementar un algoritmo de segmentación de vasos sanguíneos sobre la base de datos de imágenes DIARETDB1 que contiene 89 imágenes de fondo de ojo. Incluyendo preprocesamiento y segmentación.

Para esto es necesario tomar en cuenta la necesidad de aplicar previamente __operaciones puntuales__, __suavisado__, __operaciones morfologicas__, entre otras. Lo anterior, con la finalidad de que la entrada de nuestro algortimo de segmentación se optima y obtener como resultado segmentaciones más precisas.

# Código fuente

## Librerías

In [1]:
#manejar carpetas
import os

#Cargar bibliotecas necesarias
import numpy as np
import cv2

#graficar
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# barra de progreso
from tqdm import tqdm

from procesamiento_2 import *
from segmentacion import * 

Nota: tomar en cuenta la versión para ejecutar el notebook

In [2]:
cv2.__version__

'3.4.2'

In [3]:
def CargarArchivo(nombre):
    imagen = cv2.imread(nombre) #cargo la imagen
    imagen = cv2.resize(imagen, (512, 512)) #Escalar la imagen a múltiplos de 2
    return imagen

def GuardarArchivo(img, ruta):
    cv2.imwrite(ruta, img) 

## Procesar

Las rutas donde se obtendrán las imagenes y donde se guardaraán una vez procesadas

In [4]:
ruta   = "images/"
ruta_procesadas = "output/"+"media/" #nuevo folder por cada prueba distinta significativa

Obtener los nombres de las imágenes

In [5]:
nombres = os.listdir(ruta)
nombres = sorted(nombres)[1:90] #evito el archivo 0 llamado '.ipynb_checkpoints' al empezar en el archivo 1

In [6]:
nombres[0]

'image001.png'

Creo el ajuste de contraste CLAHE

In [7]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(3,3)) #ventana pequeña para contraste localizado

In [8]:
kernel = np.ones((3,3),np.float32)/13 #no hay razon para este kernel

## Algoritmo de Segmentación Watershed

Procedimientos:
1. Binarización
2. Es conveniente eliminar cualquier pequeño ruido blanco en la imagen. Para esto utilizamos la operación morfológica _apertura_
3. Detectar el fondo y primer plano. Para esto usamos dilatación y transformación de distancia (respectivamnte)
4. Identificar areas desconocidas. Para esto usamos el método _sustract_ con reusltado del paso 3
5. Etiquetar conocidas como indicadores
6. Marcar como 1 todos los indicadores, así el fondo no es 0, sino 1
7. Etiquetar areas desconocidas como 0
8. Aplicar Segementación Watershed sobre la imagen origial y resultado de los pasos (5 al 7) 

Esta celda será el algoritmo, luego se meterá en alguna función

In [9]:
for nombre in tqdm(nombres):
    #lectura de imagen original
    imagen = CargarArchivo(ruta+nombre)
    #cargar la capa verde de la imagen
    img_gris = imagen[:,:,1] 
    #obtener imagen suavisada
    img_blur = Media(img_gris, kernel)
    #ajusto contraste con clahe
    img_ajuste = clahe.apply(img_blur)
    #Obtener imagen binarizada inversa
    img_bin_inv = BinarizarInv(img_ajuste)
    #Apertura a la imagen binarizada
    img_opening = AperturaImagen(img_bin_inv) 
    #Segmentar
    img_seg = SegmentacionWatershed(imagen,np.uint8(img_opening))
    #guardar archivo
    GuardarArchivo(img_seg, ruta_procesadas+nombre)

100%|██████████| 89/89 [13:05<00:00,  8.82s/it]


## Justificaciones

### Filtros

El objetivo de usar filtros de suavizado es conseguir que las intensidades de los objetos pequeños se mezclen con el fondo con el fin de detectar los objetos de mayor tamaño, esto reduciendo la variación de intensidad entre pixeles vecinos. Para este problema se propusieron dos filtros: filtro de suavizado de media y filtro de suavizado Gaussiano. La forma en la que funcionan es:

 - El filtro de suavizado de media asigna al valor actual la media de la vecindad en la que se ubica. 
 - El filtro de suavizado Gaussiano modela una distribución normal en la vecindad y con eso pondera los valores, reduciendo el peso conforme se aleja del pixel actual.

Gracias a esto, el filtro de suavizado Gaussiano produce un suavizado más uniforme que el de media. Pero para la aplicación actual, necesitamos un contraste entre los objetos grandes y pequeños.

### Parametros

Los siguientes parametros fueron definidos a base de prueba y error:
- Para Media y Gaussiano con iteraciones > 1 se perdío calidad
- Para Dilatación y Apertura se probó con hasta n=25 iteración sin ver reflajado mucha diferencia a 1-2 iteraciones.

Resumen:

Método | iteraciones | kernel
-- | -- | -- | 
Media | 1 | (3x3)÷13 |
Gaussiano | 1 | k=3|
Clahe | 1 | grid=3x3| 
Dilatación| 2 | 3x3| 
Apertura | 1 | 3x3|

## Conclusiones

## Referencias

- Understanding Semantic Segmentation with UNET. (2019). Retrieved 6 February 2021, from https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
- OpenCV: Image Segmentation with Watershed Algorithm. (2021). Retrieved 5 February 2021, from https://docs.opencv.org/master/d3/db4/tutorial_py_watershed.html